# Segmenting and Clustering Neighborhoods in Toronto

In [1]:
!pip install lxml

!pip install html5lib

!pip install BeautifulSoup4

import html5lib

import lxml

     |████████████████████████████████| 5.8MB 2.8MB/s eta 0:00:01
     |████████████████████████████████| 102kB 2.9MB/s ta 0:00:011


In [2]:
import BeautifulSoup 


ModuleNotFoundError: No module named 'BeautifulSoup'

In [3]:
from bs4 import BeautifulSoup 

In [34]:
import requests
import pandas as pd 
import numpy as np

In [37]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('table',class_="wikitable sortable")
#print (table.prettify())
table_rows=table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)


df = pd.DataFrame(res, columns=["postcode", "borough", "neighbourhood"])
df

,postcode,borough,neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor
